In [ ]:
### Dependencies

# data mgmt
import pandas as pd
import numpy as np

# SQL
from sqlalchemy import create_engine
import pymysql
import os

In [ ]:
### Read csv file

df = pd.read_csv('.../employees_by_role.csv') 

In [ ]:
### Update column names and format dates

df2 = df.iloc[:,:8]
df2 = df2.rename(columns={'Employee ID':'employee_id','First Name':'first_name',
                    'Last Name':'last_name','Job Title':'job_title',
                    'Department Description':'department','Position Status':'position_status',
                    'Hire Date':'start_date','Termination Date':'end_date'})
# update datetime format
df2 = df2.assign(start_date = lambda x: pd.to_datetime(df2['start_date']))
df2 = df2.assign(end_date = lambda x: pd.to_datetime(df2['end_date']))

# df2.head() # check your work

In [ ]:
### update start and end dates
# the 'Hire Date' field in the HRIS records the start of the employees tenure
# It does not track their start date for a new position. This scrpit updates 
# start dates to more closely reflect the dates of a given position

# order records so that they are in cronological order by employee
df3 = df2.sort_values(by='end_date',na_position='last') #by end date desc
df3 = df3.sort_values(by=['employee_id','position_status'],ascending=False)

# identify rows tracking a second+ position for an employee
df3['2_plus'] = df3['employee_id'].eq(df3['employee_id'].shift())
# create a column for new position (second+) start dates
df3['new_start'] = df3['end_date'].shift()
# use original start date is this is a first position 
df3.loc[df3['2_plus'] == False, 'new_start'] = df3['start_date']
df3 = df3.drop('2_plus',axis=1)
df3 = df3[['employee_id','first_name','last_name','job_title','department','position_status','start_date','new_start','end_date']]
# remove NaN/NaTs for db import
df3 = df3.astype(object).replace(np.nan,'')

# df3.head(30) # check your work


In [ ]:
### create df of fiscal years
# this org is on a July - June fiscal year

# np array of years
array = np.arange(2009,2090,1)

# create dataframe with metadata for joins
fsc_yr = pd.DataFrame(array)
fsc_yr = fsc_yr.assign(fiscal_year = lambda x: fsc_yr[0].astype(str) +"-"+ (fsc_yr[0] - 2000 + 1).astype(str))
fsc_yr = fsc_yr.assign(start_year = lambda x: pd.to_datetime('7/1/' + fsc_yr[0].astype(str)))
fsc_yr = fsc_yr.assign(end_year = lambda x: pd.to_datetime('5/31/' + (fsc_yr[0] + 1).astype(str))) #treating May 31 as end of year given ambiguity of June hires
fsc_yr = fsc_yr[['fiscal_year','start_year','end_year']]
fsc_yr

# fsc_yr.head() # check your work

In [ ]:
### import assocaite to DB

# set up variables for db access - note this requires environmental variables
db_user = os.environ.get('USER')
db_password = os.environ.get('PASS')
db_host = os.environ.get('HOST')
db_port = 25060
db = 'employees'

#connect to DB
connection = pymysql.connect(host=db_host,
                             user=db_user,
                             password=db_password,
                             port=db_port,
                             database=db,
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

# create cursor
cursor=connection.cursor()

# create column list 
cols = "`,`".join([str(i) for i in df3.columns.tolist()])

# insert DataFrame records
for i,row in df3.iterrows():
    sql = "INSERT INTO `employees_by_role` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))

    # commit changes. connection is not autocommit by default
    connection.commit()

connection.close()

In [ ]:
### join fiscal years to individual records
# we do this using a SQL temp table because pandas does not support 
# merging on greater than/less than conditions as well as SQL

# connect to db - we need a new connection because we closed the last one after committing the employees_by_role data 
connection = pymysql.connect(host=db_host,
                             user=db_user,
                             password=db_password,
                             port=db_port,
                             database=db,
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

# create cursor
cursor=connection.cursor()

# create column list for fiscal years
cols = "`,`".join([str(i) for i in fsc_yr.columns.tolist()])

# create temp table for fiscal years
temp_sql = """
CREATE TEMPORARY TABLE "fiscal_years" (
   "key_id" bigint NOT NULL AUTO_INCREMENT,
   "fiscal_year" varchar(255) DEFAULT NULL,
   "start_year" varchar(255) DEFAULT NULL,
   "end_year" varchar(255) DEFAULT NULL,
   PRIMARY KEY ("key_id"));
"""
cursor.execute(temp_sql)
connection.commit()

# add fiscal year records to temp table
for i,row in fsc_yr.iterrows():
   yr_sql = "INSERT INTO `fiscal_years` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
   cursor.execute(yr_sql, tuple(row))
   connection.commit()

# join fiscal years onto roles metadata 
query_sql = """
SELECT 
   employee_id
 , first_name
 , last_name
 , job_title
 , department
 , position_status
 , start_date
 , new_start
 , end_date
 , fiscal_year 
FROM 
   (
   SELECT 
      employee_id
   , first_name
   , last_name
   , job_title
   , department
   , position_status
   , start_date
   , CASE WHEN 
         new_start LIKE ''
         THEN start_date
         ELSE new_start
         END AS new_start
   , end_date
   , CASE WHEN
         end_date LIKE ''
         THEN NOW()
         ELSE end_date
         END AS other_end
   FROM employees_by_role
   ) a
      LEFT JOIN 
      (
      SELECT 
         LEFT(fiscal_year,7) fiscal_year
      , start_year
      , end_year
      FROM fiscal_years 
      ) b
   ON 
   -- if new_start is during a fiscal year
      (a.new_start >= b.start_year 
      AND a.new_start <= b.end_year)
   OR 
   -- if other_end is during a fiscal year
      (a.other_end <= b.end_year
      AND a.other_end > b.start_year)
   OR 
   -- if a full fiscal year falls between new_start and other_end
      (a.new_start <= b.start_year
      AND a.other_end >= b.end_year)
"""
cursor.execute(query_sql)
roles_df = pd.DataFrame(cursor.fetchall())

# create column list for roles_df
roles_cols = "`,`".join([str(i) for i in roles_df.columns.tolist()])

# add fiscal year records to temp table
for i,row in roles_df.iterrows():
   roles_sql = "INSERT INTO `employees_by_role_by_year` (`" + roles_cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
   cursor.execute(roles_sql, tuple(row))
   connection.commit()

connection.close()

In [ ]:
### create dictionary to join department to department number
# given variablity in the strings used to identify departments
# across systems, the department numbers are needed for future 
# use of this data set

dict_map = {'department': 
        {0: 'Corporate', 1: 'Engineering', 2: 'Finance', 
        3: 'Operations', 4: 'Research', 5: 'Technology'}, 
        'dept_number': 
        {0: 1000, 1: 1001, 2: 1002, 3: 1003, 4: 1004, 5: 1005}}

df_map = pd.DataFrame.from_dict(dict_map)

# df_map.head() # check your work

In [ ]:
### join department numbers into employees_by_role_by_year

# connect to db
connection = pymysql.connect(host=db_host,
                             user=db_user,
                             password=db_password,
                             port=db_port,
                             database=db,
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

# create cursor
cursor=connection.cursor()

# create column list for department mapping
cols = "`,`".join([str(i) for i in df_map.columns.tolist()])

# create temp table for department mapping
temp_sql2 = """
CREATE TEMPORARY TABLE "dept_map" (
   "key_id" bigint NOT NULL AUTO_INCREMENT,
   "department" varchar(255) DEFAULT NULL,
   "dept_number" varchar(255) DEFAULT NULL,
   PRIMARY KEY ("key_id"));
"""
cursor.execute(temp_sql2)
connection.commit()

# add fiscal year records to temp table
for i,row in df_map.iterrows():
   map_sql = "INSERT INTO `dept_map` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
   cursor.execute(map_sql, tuple(row))
   connection.commit()

# update dept_num column to match department number
insert_sql = """
UPDATE "employees_by_role_by_year" AS a
INNER JOIN "dept_map" AS b
ON a.department = b.department
SET a.school_num = b.dept_number
WHERE a.key_id > 0
"""
cursor.execute(insert_sql)
connection.commit()

connection.close()